# Práctico 1

[Enunciado](https://github.com/DiploDatos/AprendizajeProfundo/blob/master/Practico.md) del trabajo práctico.

**Implementación de red neuronal [Perceptrón Multicapa](https://en.wikipedia.org/wiki/Multilayer_perceptron) (MLP).**

## Integrantes
- Mauricio Caggia
- Luciano Monforte
- Gustavo Venchiarutti
- Guillermo Robiglio

## Importaciones

In [1]:
import json
import gzip
import functools
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import torch
from torch.utils.data import Dataset, DataLoader, IterableDataset
from gensim.parsing import preprocessing
from gensim.models import KeyedVectors

## Constantes

In [2]:
ARCHIVO_SET_DE_ENTRENAMIENTO = './data/meli-challenge-2019/spanish.train.jsonl.gz'
ARCHIVO_SET_DE_PRUEBA = './data/meli-challenge-2019/spanish.test.jsonl.gz'
ARCHIVO_SET_DE_VALIDACION = './data/meli-challenge-2019/spanish.validation.jsonl.gz'
ARCHIVO_TOKENS = './data/meli-challenge-2019/spanish_token_to_index.json.gz'

## Carga de datos

Esta carga de datos se realiza con el fin de explorar los mismos. Otra carga de datos tendrá lugar al comento de construir el dataset.

In [3]:
%%time
file_paths = [ARCHIVO_SET_DE_ENTRENAMIENTO, # Ingresar opción 0 👁 ⚠ Tarda más de 2 minutos en cargar y puede que haya un desbordamiento de RAM o muera el kernel⚠
              ARCHIVO_SET_DE_PRUEBA,  # Ingresar opción 1
              ARCHIVO_SET_DE_VALIDACION] # Ingresar opción 2 ⚠ Tarda más de 30 segundos en cargar
i = int(input('Ingresar opción para carga de archivo (0 a 2): '))
df = pd.read_json(path_or_buf=file_paths[i], lines=True)

Ingresar opción para carga de archivo (0 a 2): 1
CPU times: user 861 ms, sys: 130 ms, total: 990 ms
Wall time: 4.43 s


In [ ]:
tokens = pd.read_json(path_or_buf=ARCHIVO_TOKENS, lines=True).T

## Análisis y visualización de los datos

El **set de entrenamiento** original tiene 4895280 registros con valores no nulos y 10 columnas. Las columnas de dicho dataset son:
- **language**: El idioma del dataset (españor o portugués). En el trabajo práctico utilizaremos solamente el dataset es español.
- **label_quality**: Calidad de la etiqueta (confiable o no confiable). Se dispone de 4508043 registros no confiables y 387237 registros confiables.
- **title**: El título que se asignó al producto. **Esta información es la que se utilizará para armar el dataser de entrenamiento.**
- **category**: La categoría que se asignó al producto. **Este es el target**.
- **split**: El tipo de dataset. _train_ para el set de entrenamiento.
- **tokenized_title**: El título tokenizado. Esto significa que los datos fueron preprocesados.
- **data**: El número asignado a cada palabra del título tokenizado.
- **target**: El número que corresponde a cada categoría.
- **n_labels**: Cantidad de etiquetas numéricas correspondientes a las distintas categorías. 632 etiquetas (0 a 631) para el caso del set de entrenamiento.
- **size**: La cantidad de registros. 4895280 registros para el caso del set de entrenamiento.

El **set de prueba** original tiene 63680 registros con valores no nulos y 10 columnas. Las columnas de dicho dataset son:
- **language**: El idioma del dataset (españor o portugués). En el trabajo práctico utilizaremos solamente el dataset es español.
- **label_quality**: Calidad de la etiqueta (confiable o no confiable). Todas las etiquetas de este dataset son confiables.
- **title**: El título que se asignó al producto.
- **category**: La categoría que se asignó al producto.
- **split**: El tipo de dataset. _test_ para el set de prueba.
- **tokenized_title**: El título tokenizado. Esto significa que los datos fueron preprocesados.
- **data**: El número asignado a cada palabra del título tokenizado.
- **target**: El número que corresponde a cada categoría.
- **n_labels**: Cantidad de etiquetas numéricas correspondientes a las distintas categorías. 632 etiquetas (0 a 631) para el caso del set de prueba.
- **size**: La cantidad de registros. 63680 registros para el caso del set de prueba.

El **set de validación** original tiene 1223820 registros con valores no nulos y 10 columnas. Las columnas de dicho dataset son:
- **language**: El idioma del dataset (españor o portugués). En el trabajo práctico utilizaremos solamente el dataset es español.
- **label_quality**: Calidad de la etiqueta (confiable o no confiable). Se dispone de 1127189 registros no confiables y 96631 registros confiables.
- **title**: El título que se asignó al producto.
- **category**: La categoría que se asignó al producto.
- **split**: El tipo de dataset. _validation_ para el set de prueba.
- **tokenized_title**: El título tokenizado. Esto significa que los datos fueron preprocesados.
- **data**: El número asignado a cada palabra del título tokenizado.
- **target**: El número que corresponde a cada categoría.
- **n_labels**: Cantidad de etiquetas numéricas correspondientes a las distintas categorías. 632 etiquetas (0 a 631) para el caso del set de validación.
- **size**: La cantidad de registros. 1223820 registros para el caso del set de validación.

El archivo **spanish_token_to_index** tiene las 50002 correspondencias que existen entre las palabras tokenizadas del título y las etiquetas numéricas bajo la columna data en los sets de entrenamiento, prueba y validación. No se utilizará este tokenizador, en lugar de ello se utilizará ...

**En este trabajo práctico se utiliza:**
- El **set de entrenamiento** para entrenar el modelo
- El **set de validación** para evaluar el modelo y ajustar hiperparámetros
- El **set de prueba** para mostrar el mejor modelo obtenido

In [4]:
df.head()

,language,label_quality,title,category,split,tokenized_title,data,target,n_labels,size
0,spanish,reliable,Mochilas Maternales Bolsos Bebe Simil Cuero Ma...,DIAPER_BAGS,test,"[mochilas, maternales, bolsos, bebe, simil, cu...","[5650, 5271, 5268, 915, 2724, 375, 37363]",318,632,63680
1,spanish,reliable,Bolso Maternal/bebe Incluye Cambiador + Correa...,DIAPER_BAGS,test,"[bolso, maternal, bebe, incluye, cambiador, co...","[502, 2742, 915, 3031, 2740, 1840, 4635]",318,632,63680
2,spanish,reliable,Mochila Maternal Land + Gancho Envio Gratis-cc,DIAPER_BAGS,test,"[mochila, maternal, land, gancho, envio, gratis]","[337, 2742, 2741, 3303, 211, 1429]",318,632,63680
3,spanish,reliable,Bolso Maternal Moderno Con Cambiador Y Correa ...,DIAPER_BAGS,test,"[bolso, maternal, moderno, cambiador, correa, ...","[502, 2742, 2983, 2740, 1840, 476, 2990]",318,632,63680
4,spanish,reliable,Bolso Maternal Moderno Con Cambiador Y Correa ...,DIAPER_BAGS,test,"[bolso, maternal, moderno, cambiador, correa, ...","[502, 2742, 2983, 2740, 1840, 4635]",318,632,63680


### Tokens y sus etiquetas

Las siguientes 3 celdas de código demuestran que la relación entre los datos bajo las columnas `tokenized_title` y `data` está dada en el archivo `spanish_token_to_index` que vincula cada palabra a un índice numérico entero. De todos modos, esto es a modo informativo.

In [ ]:
i = 0 # Un índice cualquiera para extraer datos
items = df.at[i, 'tokenized_title']
items

In [ ]:
# Comparar la salida de esta celda con la de la siguiente
df.at[i, 'data']

In [ ]:
nro_items = []
for item in items:
    id_item = tokens.loc[item][0]
    nro_items.append(id_item)
nro_items

Para el presente trabajo práctico solamente interesan las columnas **title** y **category**.

In [5]:
df.drop(columns=['language', 'label_quality', 'split', 'tokenized_title', 'data', 'target', 'n_labels', 'size'],
        inplace=True)
df.head()

,title,category
0,Mochilas Maternales Bolsos Bebe Simil Cuero Ma...,DIAPER_BAGS
1,Bolso Maternal/bebe Incluye Cambiador + Correa...,DIAPER_BAGS
2,Mochila Maternal Land + Gancho Envio Gratis-cc,DIAPER_BAGS
3,Bolso Maternal Moderno Con Cambiador Y Correa ...,DIAPER_BAGS
4,Bolso Maternal Moderno Con Cambiador Y Correa ...,DIAPER_BAGS


### Etiquetar el target.

A cada una de las 632 categorías del target se le asigna un valor entero entre 0 y 631. La relación entre la categoría u su etiqueta queda almacenada en un dataframe de Pandas llamado `df_categories`.

In [6]:
categories = df.category.unique()
sorted_categories = np.sort(categories)
df_categories = pd.DataFrame(sorted_categories, columns=['categories'])

In [7]:
df_categories['cat_tag'] = pd.DataFrame(list(range(df_categories.shape[0])))

In [8]:
df_categories.set_index('categories', inplace=True)
df_categories.head()

,cat_tag
categories,
3D_PRINTERS,0
ACCORDIONS,1
ACOUSTIC_GUITARS,2
ACTION_FIGURES,3
ADHESIVE_TAPES,4


## Construcción del Dataset

El dataset se construye a partir de un dataframe de Pandas. El mismo debe tener al menos dos columnas:
- **title**
- **category**

In [14]:
class MeLiChallengeDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform
    
    def __len__(self):
        return self.df.shape[0]

    def __getitem__(self, item):
        item = {
            "data": self.df.iloc[item]["title"],
            "target": self.df.iloc[item]["category"]
        }

        if self.transform:
            item = self.transform(item)
        
        return item

dataset = MeLiChallengeDataset(df)
i = 1000
print(f"El dataset tiene {len(dataset)} elementos.")
print(f"Elemento #{i}:\n\tData: {dataset[i]['data']}\n\tTarget: {dataset[i]['target']}")

El dataset tiene 63680 elementos.
Elemento #1000:
	Data: Jardinero Verano Beba Nena Jean Denim H/ T 6. Regalosdemama
	Target: JUMPSUITS_AND_OVERALLS


## Preprocesamiento de los datos

El preprocesamiento de texto tiene dos propósitos:
- Tokenizar los títulos (datos) de modo que se quiten los signos de puntuación y palabras cortas como preposiciones y conjunciones (stopwords), todas las palabras queden en minúsculas, se separen en listas de palabras, etc.
- Transformar las categorías en etiquetas numéricas.

In [20]:
class TextPreprocess:
    def __init__(self, df_categories, filters=None):
        if filters:
            self.filters = filters
        else:
            self.filters = [
                lambda s: s.lower(),
                preprocessing.strip_tags,
                preprocessing.strip_punctuation,
                preprocessing.strip_multiple_whitespaces,
                preprocessing.strip_numeric,
                preprocessing.remove_stopwords,
                preprocessing.strip_short,
            ]
        self.df_categories = df_categories
        
    def _preprocess_string(self, string):
        return preprocessing.preprocess_string(string, filters=self.filters)

    def __call__(self, item):
        if isinstance(item["data"], str):
            data = self._preprocess_string(item["data"])
        else:
            data = [self._preprocess_string(d) for d in item["data"]]
        
        category = item["target"]
        target = self.df_categories.at[category, 'cat_tag']
        
        return {
            "data": data,
            "target": target
        }

text_preprocess = TextPreprocess(df_categories)
print(f"Data: {text_preprocess(dataset[i])['data']}\nTarget: {text_preprocess(dataset[i])['target']}")

Data: ['jardinero', 'verano', 'beba', 'nena', 'jean', 'denim', 'regalosdemama']
Target: 327


In [ ]:
# class MeLiChallengeDataset(IterableDataset):

#     def __init__(self, path, transform=None):
#         self.dataset_path = path
#         self.transform = transform

#     def __iter__(self):
#         with gzip.open(self.dataset_path, "rt") as json_file:
#             for line in json_file:
#                 data = json.loads(line)
#                 item = {
#                     "data": data['title'],
#                     "target": data['category']
#                 }
#                 if self.transform:
#                     yield self.transform(item)
#                 else:
#                     yield item

In [ ]:
class VectorizeText:
    def __init__(self, glove_vectors_path):
        self.glove_model = KeyedVectors.load_word2vec_format("../data/glove.6B.50d.txt",
                                                             binary=False,
                                                             no_header=True)
        self.unkown_vector = np.random.randn(self.glove_model.vector_size)  # Random vector for unknown words
    
    def _get_vector(self, word):
        if word in self.glove_model:
            return self.glove_model[word]
        else:
            return self.unkown_vector
    
    def _get_vectors(self, sentence):
        return np.vstack([self._get_vector(word) for word in sentence])
    
    def __call__(self, item):
        review = []
        if isinstance(item["data"][0], str):
            review = self._get_vectors(item["data"])
        else:
            review = [self._get_vectors(d) for d in item["data"]]

        return {
            "data": review,
            "target": item["target"]
        }

In [ ]:
class WordVectorsAverage:
    def __call__(self, item):
        if item["data"][0].ndim == 2:
            data = np.vstack([np.mean(d, axis=0) for d in item["data"]])
        else:
            data = np.mean(item["data"], axis=0)
        
        return {
            "data": data,
            "target": item["target"]
        }

In [ ]:
class ToTensor:
    def __call__(self, item):
        """
        This espects a single array.
        """
        return {
            "data": torch.from_numpy(item["data"]),
            "target": torch.tensor(item["target"])
        }

In [ ]:
def compose(*functions):
    return functools.reduce(lambda f, g: lambda x: g(f(x)), functions, lambda x: x)

preprocess = TextPreprocess(df_categories)
vectorizer = VectorizeText(ARCHIVO_TOKENS)
vector_average = WordVectorsAverage()
to_tensor = ToTensor()
dataset = MeLiChallengeDataset(df,
                               transform=compose(preprocess, vectorizer, vector_average, to_tensor))

In [ ]:
for idx, sample in enumerate(dataset):
    print(sample["data"])
    print(sample["target"])
    print("=" * 50)
    
    if idx == 2:
        break

## Carga del Dataset

In [ ]:
dataloader = DataLoader(dataset, batch_size=4, shuffle=True, num_workers=0)

In [ ]:
for i_batch, sample_batched in enumerate(dataloader):
    print(i_batch, 
          sample_batched['data'].size(),
          sample_batched['target'].size())

    if i_batch == 2:
        print(sample_batched["data"])
        print(sample_batched["target"])
        break

## Construcción del Modelo

In [ ]:
class MLP(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden_layer1 = torch.nn.Linear(50, 512)
        self.hidden_layer2 = torch.nn.Linear(512, 256)
        self.output_layer = torch.nn.Linear(256, 632)
    
    def forward(self, x: torch.Tensor):
        x = self.hidden_layer1(x)  # Go through hidden layer 1
        x = torch.nn.functional.relu(x)  # Activation Function layer 1
        x = self.hidden_layer2(x) # Go through hidden layer 2
        x = torch.nn.functional.relu(x)  # Activation Function layer 2
        x = self.output_layer(x)  # Output Layer
        return x

## Algoritmo de Optimización

In [ ]:
model = MLP()
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

## Entrenamiento del modelo

In [ ]:
EPOCHS = 1
model.train()  # Tell the model to set itself to "train" mode.
for epoch in range(EPOCHS):  # loop over the dataset multiple times
    running_loss = 0.0
    # pbar = tqdm(dataloader)
    for i, data in enumerate(dataloader):
        # get the inputs; data is a list of [inputs, labels]
        # inputs: tensor de Torch de dimensión [BATCH_SIZE, 3, 32, 32]
        # labels: tensor de Torch de dimensión [BATCH_SIZE]
        inputs = data["data"]
        labels = data["target"]
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs) # Al modelo le ingreso como argumento un
                                                          # tensor de Torch de dimensión [BATCH_SIZE, 3072]
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i > 0 and i % 50 == 0:    # print every 50 mini-batches
            pbar.set_description(f"[{epoch + 1}, {i}] loss: {running_loss / 50:.4g}")
            running_loss = 0.0

## Evaluación del Modelo

## Guardado de los parámetros del modelo